## U shape PHP dynamics with multiple liquid slugs - with heat transfer

In [1]:
using Plots
pyplot()

Plots.PyPlotBackend()

In [2]:
using DifferentialEquations

In [6]:
using CartesianGrids

In [3]:
cd(dirname(pwd()))
cd("src") 


include("Systems.jl")
using ..Systems

include("Tools.jl")
using ..Tools


include("Thermomodel.jl")
using ..Thermomodel

include("Postprocessing.jl")
using ..Postprocessing

cd(dirname(pwd()))

## Parameters of the PHP

In [4]:
    ω0 = [sqrt(1.2e004), sqrt(1.2e004),sqrt(1.2e004), sqrt(1.2e004),sqrt(1.2e004), sqrt(1.2e004),sqrt(1.2e004), sqrt(1.2e004)];
    ℘ = [1.2e005, 1.2e005,1.2e005, 1.2e005,1.2e005, 1.2e005,1.2e005, 1.2e005];


     Θ = 0.15;
    θe = 1 + Θ;
    θc = 1 - Θ;
    
    He = 3000.0;
    Hc = 3000.0;
    
    X0 = 0.5;

    γ = 1.4;

In [5]:
    L = 24.0  # total length of the pipe when streched to a 1D pipe
    L2D = 2.0 # the actual length of the bended pipe in the real world
    alpha = pi/2 # inclination angle 

1.5707963267948966

## Pipe definition

In [7]:
tube = Tube(L,L2D,alpha)

Tube(24.0, 2.0, 1.5707963267948966)

## Evaporator and condenser definition

In [8]:
Xe = map(tuple, [0.0; 3.0; 7.0; 11.0; 15.0; 19.0; 23.0], [1.0; 5.0; 9.0; 13.0; 17.0; 21.0; 24.0])
evap = Evaporator(He,θe,Xe)

Evaporator(3000.0, 1.15, Tuple{Float64,Float64}[(0.0, 1.0), (3.0, 5.0), (7.0, 9.0), (11.0, 13.0), (15.0, 17.0), (19.0, 21.0), (23.0, 24.0)])

In [9]:
Xc = map(tuple, [1.0; 5.0; 9.0; 13.0; 17.0; 21.0], [3.0; 7.0; 11.0; 15.0; 19.0; 23.0])
cond = Condenser(Hc,θc,Xc)

Condenser(3000.0, 0.85, Tuple{Float64,Float64}[(1.0, 3.0), (5.0, 7.0), (9.0, 11.0), (13.0, 15.0), (17.0, 19.0), (21.0, 23.0)])

## Vapor plug and liquid slug initialization

In [10]:
X0 = [(0.5,2.5),(5.0,7.0),(7.5,8.0),(9.0,12.0),(14.0,15.0),(16.2,17.6),(20.0,21.0),(21.6,23.2)]
dXdt0 = [(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
liquids=LiquidSlug(γ,ω0,℘,X0,dXdt0)

LiquidSlug(1.4, [109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323], [120000.0, 120000.0, 120000.0, 120000.0, 120000.0, 120000.0, 120000.0, 120000.0], Tuple{Float64,Float64}[(0.5, 2.5), (5.0, 7.0), (7.5, 8.0), (9.0, 12.0), (14.0, 15.0), (16.2, 17.6), (20.0, 21.0), (21.6, 23.2)], Tuple{Float64,Float64}[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)])

In [11]:
P = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0];
vapors=VaporPlug(γ,P)

VaporPlug(1.4, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

## Construct the system

In [12]:
sys0 = PHPSystem(tube,evap,cond,liquids,vapors)

PHPSystem(Tube(24.0, 2.0, 1.5707963267948966), Evaporator(3000.0, 1.15, Tuple{Float64,Float64}[(0.0, 1.0), (3.0, 5.0), (7.0, 9.0), (11.0, 13.0), (15.0, 17.0), (19.0, 21.0), (23.0, 24.0)]), Condenser(3000.0, 0.85, Tuple{Float64,Float64}[(1.0, 3.0), (5.0, 7.0), (9.0, 11.0), (13.0, 15.0), (17.0, 19.0), (21.0, 23.0)]), LiquidSlug(1.4, [109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323, 109.54451150103323], [120000.0, 120000.0, 120000.0, 120000.0, 120000.0, 120000.0, 120000.0, 120000.0], Tuple{Float64,Float64}[(0.5, 2.5), (5.0, 7.0), (7.5, 8.0), (9.0, 12.0), (14.0, 15.0), (16.2, 17.6), (20.0, 21.0), (21.6, 23.2)], Tuple{Float64,Float64}[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]), VaporPlug(1.4, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]))

In [13]:
typeof(sys0)

PHPSystem

## Heat diffusivity

In [14]:
α = 1;

## Construct U for wall heat function

In [52]:
function ifamongone(value::Float64, range::Tuple{Float64,Float64})
    return (value > range[1]) && (value < range[2]) ? true : false
end

ifamongone (generic function with 1 method)

In [67]:
function ifamong(value::Float64, X::Array{Tuple{Float64,Float64},1})
    
    return Bool(sum(ifamongone.(value,X)))
end

ifamong (generic function with 3 methods)

In [68]:
ifamong(0.6, sys0.evaporator.Xe)

true

In [73]:
ifamong(0.6, sys0.condenser.Xc)

false

In [76]:
function settemperature!(Θᵣ,x,sys0)
               
    Θᵣ = 1;
    
    if ifamong(xvalue, sys0.evaporator.Xe)
        Θᵣ = sys0.evaporator.Θe
    elseif ifamong(xvalue, sys0.condenser.Xc)
        Θᵣ = sys0.condenser.Θc
        else Θᵣ = 1
    end
    
    return Θᵣ
            end

Foo (generic function with 1 method)

In [80]:
settemperature!(Θᵣ,x,sys0)

MethodError: MethodError: no method matching length(::PHPSystem)
Closest candidates are:
  length(!Matched::Core.SimpleVector) at essentials.jl:593
  length(!Matched::Base.MethodList) at reflection.jl:849
  length(!Matched::Core.MethodTable) at reflection.jl:923
  ...

In [78]:
x = Nodes(Dual,(100,1))
u0[50]
# 

0.0

In [24]:
laplacian(u0)

Nodes{Dual,100,1,Float64,Array{Float64,2}} data
Printing in grid orientation (lower left is (1,1))
1×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## Solve the constructed ODE for U

In [37]:
tstep=1.0e-5

p = sys0
tspan = (0.0, 0.05);
prob = ODEProblem(zhang2002model!, u, tspan, p)
sol = solve(prob, RK4(), adaptive=false, dt=tstep);
# sol = solve(prob, RK4(), adaptive=false, dt=tstep, callback=cb);